In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define el modelo CNN
model = models.Sequential()

# Primera capa convolucional50
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(50, 50, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Segunda capa convolucional
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# tercera capa convolucional
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# 4 capa convolucional
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))



# Flatten Layer : permite concatenar las caracteristicas en un vector unidimensional (aplanar)
#sirve para preparar los datos de entrada a una red completamente conectada (FC - fully connected)
model.add(layers.Flatten())

# Fully Connected Layer
model.add(layers.Dense(256, activation='relu'))

# Capa de salida
model.add(layers.Dense(1, activation='softmax'))  # En este caso se tienen 10 clases 

# Compile el modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# mostar el resumen
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 64)         0

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Definir los parametros
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Cargar y procesar sus datos (Asumiendo que los tiene en un directorio)
train_generator = datagen.flow_from_directory(
      'C:/Users/paola/OneDrive/Escritorio/histopatologia/train',
    target_size=(50, 50),  # Ajustar
    batch_size=1024,           # Ajustar
    class_mode='binary' # Depende de la tarea (binary , categorical classification)
)


datagen_test = ImageDataGenerator()
test_generator = datagen_test.flow_from_directory(
     'C:/Users/paola/OneDrive/Escritorio/histopatologia/test',
    target_size=(50, 50),  # Ajustar
    batch_size=1024,           # Ajustar
    class_mode='binary' # Depende de la tarea (binary , categorical classification)
)


Found 194001 images belonging to 2 classes.
Found 83523 images belonging to 2 classes.


In [ ]:
# entrenar su modelo
model.fit(train_generator, epochs=5)

Epoch 1/5
  7/190 [>.............................] - ETA: 1:03:41 - loss: 3.0820 - accuracy: 0.7122

In [ ]:
_, accuracy = model.evaluate(x=test_generator)
print('Accuracy: %.2f' % (accuracy*100))

In [16]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings

# Suprimir las advertencias para UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UserWarning)

print("[INFO] evaluating network...")
test_generator.reset()
predIdxs = model.predict(x=test_generator)
predIdxs = np.argmax(predIdxs, axis=1)

# Calcular el informe de clasificación con zero_division
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    class_report = classification_report(test_generator.classes, predIdxs, target_names=test_generator.class_indices.keys())

print(class_report)


[INFO] evaluating network...
82/82 [==============================] - 48s 583ms/step
              precision    recall  f1-score   support

      cancer       0.28      1.00      0.44     23770
        sano       0.00      0.00      0.00     59753

    accuracy                           0.28     83523
   macro avg       0.14      0.50      0.22     83523
weighted avg       0.08      0.28      0.13     83523

